# Load SARS-CoV-2 Virus Strain Data from CNCB
**[Work in progress]**

This notebook downloads and standardizes viral strain and variation data from CNCB for ingestion into a Knowledge Graph.

Data source: [China National Center for Bioinformation, 2019 Novel Coronavirus Resource (2019nCoVR)](https://bigd.big.ac.cn/ncov/release_genome)

Author: Peter Rose (pwrose@ucsd.edu)

In [1]:
import os
import pandas as pd
import dateutil
import re
from pathlib import Path
import glob
#from os import path
import ftplib

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
# Path will take care of handling operating system differences.
NEO4J_HOME = Path(os.getenv('NEO4J_HOME'))
print(NEO4J_HOME)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3


In [4]:
# Create a directory to cache variation data
CACHE = Path(NEO4J_HOME / 'import/cache')
CACHE.mkdir(exist_ok=True)

## Download SARS-CoV-2 Strain metadata

This notebook will download > 17,000 files. To work on a small sample (50 files), set 

`run_small_sample_only = True`

In [5]:
run_small_sample_only = False

In [6]:
metadata_url = "https://bigd.big.ac.cn/ncov/genome/export/meta"
annotation_url = "ftp://download.big.ac.cn/GVM/Coronavirus/gff3/" 

In [7]:
df = pd.read_excel(metadata_url, dtype='str')
df.fillna('', inplace=True)

In [8]:
print("Total number of strains:", df.shape[0])

Total number of strains: 32706


In [9]:
df = df.query("`Sequence Quality` == 'High'")
df = df.query("`Nuc.Completeness` == 'Complete'")

In [10]:
print("Number of complete high quality strains", df.shape[0])

Number of complete high quality strains 15861


In [11]:
df.head()

,Virus Strain Name,Accession ID,Data Source,Related ID,Nuc.Completeness,Sequence Length,Sequence Quality,Quality Assessment,Host,Sample Collection Date,Location,Originating Lab,Submission Date,Submitting Lab,Create Time,Last Update Time
0,BetaCoV/Wuhan/HBCDC-HB-01/2019,NMDC60013088-01,NMDC,EPI_ISL_402132,Complete,29848,High,0/0/0/1/NO,Homo sapiens,2019-12-30,China / Hubei,Hubei Provincial Center for Disease Control an...,2020-01-19,Hubei Provincial Center for Disease Control an...,2020-01-20 20:04:48,2020-05-07 23:03:25
1,BetaCoV/Nonthaburi/74/2020,EPI_ISL_403963,GISAID,,Complete,29859,High,0/0/0/0/NO,Homo sapiens,2020-01-13,Thailand/ Nonthaburi Province,"Department of Medical Sciences, Ministry of Pu...",2020-01-17,"Department of Medical Sciences, Ministry of Pu...",2020-01-20 20:04:48,2020-01-20 20:04:48
2,BetaCoV/Nonthaburi/61/2020,EPI_ISL_403962,GISAID,,Complete,29848,High,0/0/0/0/NO,Homo sapiens,2020-01-08,Thailand/ Nonthaburi Province,"Department of Medical Sciences, Ministry of Pu...",2020-01-17,"Department of Medical Sciences, Ministry of Pu...",2020-01-20 20:04:48,2020-01-20 20:04:48
3,BetaCoV/Wuhan/IVDC-HB-04/2020,NMDC60013085-01,NMDC,EPI_ISL_402120,Complete,29896,High,0/0/0/2/NO,Homo sapiens,2020-01-01,China / Hubei / Wuhan,National Institute for Viral Disease Control a...,2020-01-11,National Institute for Viral Disease Control a...,2020-01-20 20:04:48,2020-05-07 23:03:25
4,BetaCoV/Wuhan/IVDC-HB-01/2019,NMDC60013084-01,NMDC,EPI_ISL_402119,Complete,29891,High,0/0/0/0/NO,Homo sapiens,2019-12-30,China / Hubei / Wuhan,National Institute for Viral Disease Control a...,2020-01-10,National Institute for Viral Disease Control a...,2020-01-20 20:04:48,2020-05-07 23:03:25


#### Create a separate row for each Accession and Related ID

In [12]:
df['Accession ID'] = df['Accession ID'].str.strip()
df['Related ID'] = df['Related ID'].str.strip()

# combine all ids into a single column
df['alias'] = df['Accession ID'] + df['Related ID'].apply(lambda s: ',' + s if len(s) > 0 else s)
df['alias'] = df['alias'].str.replace(' ', '')

# then "explode" ids into separate rows
df['id'] = df['alias'].apply(lambda s: s.split(','))
df = df.explode('id')
df['id'] = df['id'].str.strip()
df['alias'] = df['alias'].str.replace(',', ';')

#### Assign taxonomy ids

In [13]:
# TODO a find general solution to map host name to NCBI taxonomy**

# some organism specifications are ambiguous, 
# they don't match a specificn NCBI taxonomy

taxonomy_to_id = {'Human': 'taxonomy:9606', 
                  'Homo sapiens': 'taxonomy:9606',
                  'Homo Sapiens': 'taxonomy:9606',
                  'Rhinolophus affinis': 'taxonomy:59477', 
                  'Rhinolophus malayanus': 'taxonomy:608659', 
                  'Mustela lutreola': 'taxonomy:9666',
                  'Panthera tigris jacksoni': 'taxonomy:419130',
                  'Rhinolophus sp. (bat)': 'taxonomy:49442', # ambiguous
                  'bat': 'taxonomy:49442', # ambiguous
                  'Manis javanica': 'taxonomy:9974',
                  'palm civet': 'taxonomy:71116', # ambiguous
                  'Canine': 'taxonomy:9608', # ambiguous
                  'canine': 'taxonomy:9608', # ambiguous
                  'Felis catus': 'taxonomy:9685'
                 }

In [14]:
# assign taxonomy id to host
df['hostTaxonomyId'] = df['Host'].apply(lambda s: taxonomy_to_id.get(s, s))
df['hostTaxonomyId'].unique()

array(['taxonomy:9606', 'taxonomy:59477', 'Environment', 'taxonomy:9974',
       'taxonomy:608659', 'taxonomy:419130', 'taxonomy:9666',
       'taxonomy:9608', 'Homo Sapiens '], dtype=object)

In [15]:
df['taxonomyId'] = 'taxonomy:2697049' # SARS-CoV-2

#### Standardize node property names (CURIEs and URIs)

In [16]:
df.rename(columns={'Virus Strain Name': 'name',
                   'Sample Collection Date':'collectionDate',
                   'Location':'location'}, 
          inplace=True)

In [17]:
# https://registry.identifiers.org/registry/insdc
insdc_pattern = re.compile('^([A-Z]\d{5}|[A-Z]{2}\d{6}|[A-Z]{4}\d{8}|[A-J][A-Z]{2}\d{5})(\.\d+)?$')

In [18]:
def assign_curie(id):
    id = id.strip()
    # remove underscore to enable CURIE matching of NCBI reference sequences NC_...
    id = id.replace('NC_', 'NC') 
    if len(id) > 0:
        if id.startswith('EPI'):
            return 'https://www.gisaid.org/' + id
        elif id.startswith('NC_'):
            # NCBI reference sequences resolve with ncbiprotein CURIE
            return 'ncbiprotein:' + id
        elif insdc_pattern.match(id) != None:
            return 'insdc:' + id
        else:
            # TODO are URIs available for these cases?
            return id
    else:
        return id

In [19]:
strains = df[['id', 'name', 'alias', 'taxonomyId', 'hostTaxonomyId','collectionDate', 'location']].copy()
strains['id'] = strains['id'].apply(assign_curie)
strains.head()

,id,name,alias,taxonomyId,hostTaxonomyId,collectionDate,location
0,NMDC60013088-01,BetaCoV/Wuhan/HBCDC-HB-01/2019,NMDC60013088-01;EPI_ISL_402132,taxonomy:2697049,taxonomy:9606,2019-12-30,China / Hubei
0,https://www.gisaid.org/EPI_ISL_402132,BetaCoV/Wuhan/HBCDC-HB-01/2019,NMDC60013088-01;EPI_ISL_402132,taxonomy:2697049,taxonomy:9606,2019-12-30,China / Hubei
1,https://www.gisaid.org/EPI_ISL_403963,BetaCoV/Nonthaburi/74/2020,EPI_ISL_403963,taxonomy:2697049,taxonomy:9606,2020-01-13,Thailand/ Nonthaburi Province
2,https://www.gisaid.org/EPI_ISL_403962,BetaCoV/Nonthaburi/61/2020,EPI_ISL_403962,taxonomy:2697049,taxonomy:9606,2020-01-08,Thailand/ Nonthaburi Province
3,NMDC60013085-01,BetaCoV/Wuhan/IVDC-HB-04/2020,NMDC60013085-01;EPI_ISL_402120,taxonomy:2697049,taxonomy:9606,2020-01-01,China / Hubei / Wuhan


In [20]:
strains.to_csv(NEO4J_HOME / "import/01d-CNCBStrain.csv", index=False)

## Merge Metadata with Variation Data

#### Get list of file names from FTP site

In [21]:
server = "download.big.ac.cn"
user = "anonymous"
password = "anonymous"
source = "/GVM/Coronavirus/gff3/"

ftp = ftplib.FTP(server)
ftp.login(user, password)
ftp.cwd(source) 
filelist=ftp.nlst()
ftp.quit()

'221 Goodbye.'

In [22]:
df_file = pd.DataFrame(filelist, columns=['filename'])

Extract identifiers from file name

Example: 2019-nCoV_CNA0013697_variants.gff3 -> CNA0013697

In [23]:
df_file['id'] = df_file['filename'].str[10:]
df_file['id'] = df_file['id'].str.replace('_variants.gff3','')

In [24]:
df_file = df_file.query("id != ''")

In [25]:
print("Number of available files:", df_file.shape[0])

Number of available files: 15919


In [26]:
df_file.head()

,filename,id
0,2019-nCoV_CNA0007332_variants.gff3,CNA0007332
1,2019-nCoV_CNA0007334_variants.gff3,CNA0007334
2,2019-nCoV_CNA0007335_variants.gff3,CNA0007335
3,2019-nCoV_CNA0013697_variants.gff3,CNA0013697
4,2019-nCoV_CNA0013698_variants.gff3,CNA0013698


In [27]:
df = df.merge(df_file, on='id')

In [28]:
print('Strains with a matching filename:', df.shape[0])

Strains with a matching filename: 15848


In [29]:
if run_small_sample_only:
    df = df.sample(n=50, random_state=5)

In [30]:
df.head()

,name,Accession ID,Data Source,Related ID,Nuc.Completeness,Sequence Length,Sequence Quality,Quality Assessment,Host,collectionDate,location,Originating Lab,Submission Date,Submitting Lab,Create Time,Last Update Time,alias,id,hostTaxonomyId,taxonomyId,filename
5499,SARS-CoV-2/human/USA/OR-UW-4388/2020,MT345878,GenBank,EPI_ISL_426076,Complete,29868,High,"0/0/1/10/29866~29870(5-3-0.60,Deletion:29866-2...",Homo Sapiens,2020-03-29,United States / Oregon,UW Virology Lab,2020-04-14,UW Virology Lab,2020-04-15 12:42:16,2020-04-17 17:02:44,MT345878;EPI_ISL_426076,EPI_ISL_426076,taxonomy:9606,taxonomy:2697049,2019-nCoV_EPI_ISL_426076_variants.gff3
399,hCoV-19/France/HF1795/2020,EPI_ISL_414627,GISAID,,Complete,29897,High,0/0/0/8/NO,Homo sapiens,2020-03-02,France / Hauts de France / Compiègne,National Reference Center for Viruses of Respi...,2020-03-15,National Reference Center for Viruses of Respi...,2020-03-16 13:36:12,2020-03-16 13:36:12,EPI_ISL_414627,EPI_ISL_414627,taxonomy:9606,taxonomy:2697049,2019-nCoV_EPI_ISL_414627_variants.gff3
12396,hCoV-19/USA/NY-NYUMC521/2020,EPI_ISL_444638,GISAID,,Complete,29903,High,0/0/0/8/NO,Homo Sapiens,2020-03-30,United States / New York / Queens,"Departments of Pathology and Medicine, New Yor...",2020-05-14,"Departments of Pathology and Medicine, New Yor...",2020-05-15 12:39:58,2020-05-15 12:39:58,EPI_ISL_444638,EPI_ISL_444638,taxonomy:9606,taxonomy:2697049,2019-nCoV_EPI_ISL_444638_variants.gff3
967,hCoV-19/USA/WA-S20/2020,EPI_ISL_417073,GISAID,,Complete,29903,High,0/0/0/6/NO,Homo sapiens,2020-03-02,United States / Washington,Seattle Flu Study,2020-03-24,Seattle Flu Study,2020-03-25 11:59:51,2020-03-25 11:59:51,EPI_ISL_417073,EPI_ISL_417073,taxonomy:9606,taxonomy:2697049,2019-nCoV_EPI_ISL_417073_variants.gff3
10885,hCoV-19/Austria/CeMM0220/2020,EPI_ISL_438054,GISAID,,Complete,29866,High,"0/1/0/10/28881~28883(3-3-1.00,SNP:28881; SNP:2...",Homo Sapiens,2020-03-16,Austria / Vienna,"Bergthaler laboratory, CeMM Research Center fo...",2020-05-12,"Bergthaler laboratory, CeMM Research Center fo...",2020-05-12 15:00:57,2020-05-12 15:00:57,EPI_ISL_438054,EPI_ISL_438054,taxonomy:9606,taxonomy:2697049,2019-nCoV_EPI_ISL_438054_variants.gff3


Keep only unique entries (there are a few duplicate cases)

In [31]:
df.drop_duplicates(subset='name', inplace=True)
df.drop_duplicates(subset='id', inplace=True)

In [32]:
print('Strains with a matching filename:', df.shape[0])

Strains with a matching filename: 50


#### Download variant annotations for each strain
To avoid download the same files every time, they are cached, and newly downloaded files are added to the cache.

In [33]:
names = ['taxon1', 'variantType', 'name', 'start', 'end','x1', 'x2', 'x3','taxon2', 'x4', 'strainStart', 'taxon3', 'x5', 'strainEnd', 'ref', 'alt', 'vepAnnotation']

In [34]:
def download_gff3(filename, url):
    gff3 = pd.read_csv(url, header=None, comment='#', sep='[\t|;]', engine='python', names=names)
    gff3['ref'] = gff3['ref'].str.replace('REF=','')
    gff3['alt'] = gff3['alt'].str.replace('ALT=','')
    gff3['vepAnnotation'] = gff3['vepAnnotation'].str.replace('VEP=','')
    # prepare for 3-way split (need at least two commas)
    gff3['vepAnnotation'] = gff3['vepAnnotation'].apply(lambda s: s + ',,' if s.count(',') < 2 else s)
    # 3-way split
    gff3[['variantConsequence','proteinVariant','geneVariant']] = gff3['vepAnnotation'].str.split(',', n=2, expand=True)
    gff3['geneVariant'] = gff3['geneVariant'].str.replace('gene-','')
    gff3 = gff3[['name', 'variantType', 'start', 'end', 'ref', 'alt', 'variantConsequence', 'proteinVariant', 'geneVariant']]
    
    filename = row['filename'] + '.csv'
    gff3.to_csv(CACHE / filename, index=False)

In [35]:
for index, row in df.iterrows():
    url = annotation_url + row['filename']
    filename = row['filename'] + '.csv'
    if not Path.exists(CACHE / filename):
        try:
            download_gff3(row['filename'], url)
            print(row['filename'], end=' ')
        except:
            print('Download failed for: ', row['filename'])

2019-nCoV_EPI_ISL_426076_variants.gff3 2019-nCoV_EPI_ISL_414627_variants.gff3 2019-nCoV_EPI_ISL_444638_variants.gff3 2019-nCoV_EPI_ISL_417073_variants.gff3 2019-nCoV_EPI_ISL_438054_variants.gff3 2019-nCoV_EPI_ISL_445118_variants.gff3 2019-nCoV_EPI_ISL_436452_variants.gff3 2019-nCoV_EPI_ISL_434217_variants.gff3 2019-nCoV_EPI_ISL_441774_variants.gff3 2019-nCoV_EPI_ISL_414561_variants.gff3 2019-nCoV_EPI_ISL_420789_variants.gff3 2019-nCoV_EPI_ISL_420180_variants.gff3 2019-nCoV_EPI_ISL_437428_variants.gff3 2019-nCoV_EPI_ISL_437404_variants.gff3 2019-nCoV_EPI_ISL_432925_variants.gff3 2019-nCoV_EPI_ISL_438875_variants.gff3 2019-nCoV_EPI_ISL_421484_variants.gff3 2019-nCoV_EPI_ISL_422534_variants.gff3 2019-nCoV_EPI_ISL_451370_variants.gff3 2019-nCoV_EPI_ISL_419570_variants.gff3 Download failed for:  2019-nCoV_EPI_ISL_447257_variants.gff3
2019-nCoV_EPI_ISL_438901_variants.gff3 2019-nCoV_EPI_ISL_433953_variants.gff3 2019-nCoV_EPI_ISL_439191_variants.gff3 2019-nCoV_EPI_ISL_451216_variants.gff3 201

### Concatenate all variation data into a single dataframe

In [36]:
# use all cached data files
path = str(CACHE / '*.csv')
filenames = glob.glob(path)

# list of dataframes
dfs = []

for filename in filenames:
    df = pd.read_csv(filename, index_col=None, header=0)
    dfs.append(df)

variations = pd.concat(dfs, axis=0, ignore_index=True)
variations.fillna('', inplace=True)

List of variant types and consequences:

https://uswest.ensembl.org/info/genome/variation/prediction/classification.html

https://uswest.ensembl.org/info/genome/variation/prediction/predicted_data.html#consequences

#### Extract protein position and protein id from proteinVariant string

Example: QHD43415.1:p.5828P>L

proteinPosition: 5828
proteinId: QHD43415

In [37]:
position_pattern = re.compile(':p\.(.*?)[A-Z|\-]+')

In [38]:
def extract_protein_position(s):
    if s == '':
        return s
    else:
        groups = position_pattern.search(s)
        if groups == None:
            return ''
        else:
            return groups.group(1)

In [39]:
variations['proteinPosition'] = variations['proteinVariant'].apply(extract_protein_position)

In [40]:
variations['proteinId'] = variations['proteinVariant'].apply(lambda s: s.split('.')[0] if '.' in s else '')

In [41]:
variations['proteinId'].unique()

array(['', 'QHD43415', 'QHD43416', 'QHD43417', 'QHD43423', 'QHD43422',
       'QHI42199', 'QHD43419', 'QHD43420', 'QHD43421', 'QHD43418'],
      dtype=object)

#### Assign SARS-CoV-2 taxonomy id

In [42]:
variations['taxonomyId'] = 'taxonomy:2697049'

#### Assign Reference genome

The first SARS-CoV-2 genome sequence is the reference for the variant annotation below.

[Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1](https://www.ncbi.nlm.nih.gov/nuccore/MN908947)

In [43]:
variations['referenceGenome'] = 'insdc:MN908947'

In [44]:
print('variantType:', variations['variantType'].unique())

variantType: ['Deletion' 'SNP' 'Insertion' 'Indel']


In [45]:
print("variantConsequence:", variations['variantConsequence'].unique())

variantConsequence: ['intergenic_variant' 'upstream_gene_variant' 'missense_variant'
 'synonymous_variant' 'downstream_gene_variant' 'inframe_insertion'
 'coding_sequence_variant' 'inframe_deletion' 'stop_gained']


In [46]:
print("Number of variants:", variations.shape[0])

Number of variants: 1263


In [47]:
variations.to_csv(NEO4J_HOME / "import/01d-CNCBVariant.csv", index=False)

In [48]:
variations.head()

,name,variantType,start,end,ref,alt,variantConsequence,proteinVariant,geneVariant,proteinPosition,proteinId,taxonomyId,referenceGenome
0,hCoV-19/USA/NY-PV09028/2020,Deletion,1,64,AATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCG...,-,intergenic_variant,,,,,taxonomy:2697049,insdc:MN908947
1,hCoV-19/USA/NY-PV09028/2020,SNP,241,241,C,T,upstream_gene_variant,DISTANCE=25,"QHD43415.1,orf1ab",,,taxonomy:2697049,insdc:MN908947
2,hCoV-19/USA/NY-PV09028/2020,SNP,1059,1059,C,T,missense_variant,QHD43415.1:p.265T>I,orf1ab:c.794aCc>aTc,265,QHD43415,taxonomy:2697049,insdc:MN908947
3,hCoV-19/USA/NY-PV09028/2020,SNP,3037,3037,C,T,synonymous_variant,QHD43415.1:p.924F,orf1ab:c.2772ttC>ttT,924,QHD43415,taxonomy:2697049,insdc:MN908947
4,hCoV-19/USA/NY-PV09028/2020,SNP,10851,10851,C,T,missense_variant,QHD43415.1:p.3529A>V,orf1ab:c.10586gCt>gTt,3529,QHD43415,taxonomy:2697049,insdc:MN908947
